In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['DDE_BACKEND'] = 'pytorch'

In [2]:
import deepxde as dde 
import numpy as np 

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.
/userhome/jeon_mg/miniforge3/envs/pinn_torch/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [3]:
layer_size = [3] + [256] * 8 + [3]
activation = 'sin'
initializer = 'Glorot normal'
net = dde.nn.FNN(layer_size, activation, initializer)
model = dde.Model(None, net)
model.compile('adam', lr=5e-4)

Compiling model...
'compile' took 0.353755 s



In [ ]:
model.restore('model3/nf2-99000.pt')